Übung: Blue/Green Deployments
-----------------

In dieser Übung führen wir einen Blue/Green Deployment durch.

Dazu werden zwei Deployments von Webshop erstellt und mittels Veränderung der Zusteuerung des Service zwischen den Version hin und her geschaltet.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [ ]:
! kubectl create namespace bluegreen

Deployment und Services wurde in einzelne Dateien aufgeteilt. Die Ingress Ressource, aus Einfachheitsgründen weggelassen.

Die Labels und das Image wurden um eine Version Angabe erweitert:

    apiVersion: v1
    kind: Service
    metadata:
      name: webshop
      labels:
        app: webshop
    spec:
      ...
      selector:
        app: webshop
        version: "1.0.0"    
        
- - -

    apiVersion: apps/v1
    kind: Deployment
    metadata:
      name: webshop
      version: 1.0.0
    ...
    spec:
      containers:
      - name: webshop
        image: registry.gitlab.com/mc-b/misegr/misegr/webshop:1.0.0
        imagePullPolicy: IfNotPresent       



In [ ]:
! kubectl apply -f 09-4-Deployment/webshop-deployment-blue.yaml --namespace bluegreen
! kubectl apply -f 09-4-Deployment/webshop-service-blue.yaml --namespace bluegreen

Das Ergebnis unterscheidet sich, vorerst, nicht von einem normalen Deployment

In [ ]:
! kubectl get all --namespace bluegreen
! echo "http://$(cat ~/work/server-ip):$(kubectl get service --namespace bluegreen webshop -o=jsonpath='{ .spec.ports[0].nodePort }')"

- - -

Für das **green** Deployment erstellen wir nur ein Deployment aber kein Service:

In [ ]:
! kubectl apply -f 09-4-Deployment/webshop-deployment-green.yaml --namespace bluegreen
! kubectl get all --namespace bluegreen -o wide

Der Service steuert auf die **green** Pods zu.

Erst nach Ändern des selectors auf die **green** Version wird diese für den Anwender sichtbar

        apiVersion: v1
        kind: Service
        metadata:
          name: webshop
          labels:
            app: webshop
        spec:
          ...
          selector:
            app: webshop
            version: "2.0.0"
        

In [ ]:
! kubectl apply -f 09-4-Deployment/webshop-service-green.yaml --namespace bluegreen

An den Anzahl Ressourcen ändert der Selector nichts

In [ ]:
! kubectl get all --namespace bluegreen -o wide

- - -
Nach den erfolgreichen Tests der neuen Version können wir das **blue** Deployment löschen.

In [ ]:
! kubectl delete -f 09-4-Deployment/webshop-deployment-blue.yaml --namespace bluegreen
! kubectl get all --namespace bluegreen

- - -

Aufräumen

In [ ]:
! kubectl delete namespace bluegreen